In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize,LogNorm
import matplotlib.ticker as ticker
from matplotlib.cm import ScalarMappable
savefig = False
import importlib
import tango_model as tango_model
import sys
importlib.reload(sys.modules['tango_model'])
from tango_model import *
import scipy as sp
import torch
import networkx as nx

# Import the degree distribution CCDF from SEXUS
degree_dist_ccdf_sexus = np.genfromtxt('Data/Degree_dist_CCDF_SEXUS.csv', delimiter=',', skip_header=1)

In [3]:
N           = 85000
M           = 28
T           = 365*15
r_old_mean  = 0.2
pexp        = 1.55
lb          = 0.15
ub          = 70
window      = 365
old_dist    = "gamma"
alpha       = 2
T_track     = 365*10
n_couples   = N

simulation = EpidemicSimulation(N,M,r_old_mean,pexp,old_dist,alpha,ub,lb)

# Initialize peacetime parameters and run the simulation
simulation.initialize_peace_time(T=T, Twindow=window, progress=True, all_conn=False,T_track=T_track,n_couples=n_couples)
simulation.run_peace_time_simulation()

  3%|▎         | 170/5475 [00:09<04:55, 17.96it/s]


KeyboardInterrupt: 

In [ ]:
durations                   = np.array(simulation.get_partnership_durations(include_non_steady=True))
num_partnerships            = len(durations)
tot_encounters,tot_partners = simulation.analyze_encounters()
print(f"Number of partnerships: {num_partnerships}")

In [ ]:
active_mask         = tot_partners >= 1 
steadymask          = simulation.get_steady_partners_mask()

#print(f"Mean  and std number of partners:                                   {(tot_partners[active_mask]).float().mean():.2f} +- {(tot_partners[active_mask]).float().std():.2f} pr yr (SEXUS: 5.7 +- 9.2)")
print(f"Mean rate of encounters:                                            {(tot_encounters[active_mask]).float().mean()/52:.2f} pr wk (SEXUS: 0.74)")
print(f"Median rate of encounters:                                          {(tot_encounters[active_mask]).float().median()/52:.2f} pr wk (SEXUS: 0.5)")
#print(f"Fraction of people with exactly one partner the last year:          {(tot_partners == 1).sum()/(active_mask).sum():.2f} (SEXUS 0.41)")
print(f"Fraction of inactive:                                               {(tot_partners == 0).sum()/N:.2f} (SEXUS 0.12)")
print("")

print(f"Steady fraction:                                                    {sum(simulation.get_steady_partners_mask())/active_mask.sum():.2f}    (SEXUS 0.26)") #Don't divide by two, since we are counting members, not partnerships
print(f"Probability of relationship lasting a year:                         {(durations > 365).sum()/num_partnerships:.2f}    (SEXUS: 0.33)")

print("")

print(f"Mean number of yrly partners for members with partner:              {(tot_partners[active_mask &  steadymask]).float().mean():.2f} (SEXUS: 5.0 - (excluding partnerships newer than a year))")
print(f"Mean number of yrly partners for members without partner:           {(tot_partners[active_mask & ~steadymask]).float().mean():.2f} (SEXUS: 5.8)")
print("")
print(f"Mean  r_new among inactive:                                         {(simulation.r_new[~active_mask]).float().mean()*365:.2f} pr yr (total: {(simulation.r_new).float().mean()*365:.2f})")
print(f"Mean  r_old among inactive:                                         {(simulation.r_old[~active_mask]).float().mean()*7:.2f} pr wk (total: {(simulation.r_old).float().mean()*7:.2f})")

# Extract the active partners
active_partners = tot_partners[active_mask].numpy()

# Step 1: Define the specific x-axis values
x_values = np.array([1, 2, 3, 5, 10, 20, 50])

# Step 2: Calculate CCDF for only these x-axis values
ccdf = []
n = len(active_partners)
for value in x_values:
    # Probability that a value is greater than or equal to `value`
    prob = np.sum(active_partners >= value) / n
    ccdf.append(prob)

# Step 3: Plot CCDF for selected x-values
plt.figure(figsize=(8, 6))
plt.plot(x_values, ccdf, label='Simulation', marker='o')
plt.plot(x_values,degree_dist_ccdf_sexus, marker='o',label = "SEXUS")
plt.xticks(x_values)  # Show only the selected x-axis values
plt.xlabel('Active Partners')
plt.ylabel('P(X ≥ x)')
plt.xscale("log")
plt.yscale("log")
plt.title('Complementary Cumulative Distribution Function (CCDF)')
plt.grid(True,which = "both")
plt.legend()
plt.show()

In [ ]:
simulation.plot_sanity_check()

# Understanding the power law exponent

In [ ]:
# Does the KDist distribution itself give a different exponent, due to the low lower bound?
test_distribution = simulation.Kdist()
plt.hist(test_distribution[test_distribution > 1],bins = 100,label = "Simulation")
plt.yscale("log")
plt.xscale("log")
height = 5000
plt.plot([1,80],[height,height*80**-1.8],label = r"$\gamma = 1.8$")
plt.plot([1,80],[height,height*80**-1.55],label = r"$\gamma = 1.5$")
plt.legend()

In [ ]:
# Does the changed exponent continue farther up in the tail?

x_values2 = np.array([1, 2, 3, 5, 10, 20, 50,100,200])

# Step 2: Calculate CCDF for only these x-axis values
ccdf2 = []
n = len(active_partners)
for value in x_values2:
    # Probability that a value is greater than or equal to `value`
    prob = np.sum(active_partners >= value) / n
    ccdf2.append(prob)

# Step 3: Plot CCDF for selected x-values
plt.figure(figsize=(8, 6))
plt.plot(x_values2, ccdf2, label='Simulation', marker='o')
#plt.plot(x_values2,degree_dist_ccdf_sexus, marker='o',label = "SEXUS")
plt.plot([1,200],[1,200**(-0.8)])
plt.xticks(x_values2)  # Show only the selected x-axis values
plt.xlabel('Active Partners')
plt.ylabel('P(X ≥ x)')
plt.xscale("log")
plt.yscale("log")
plt.title('Complementary Cumulative Distribution Function (CCDF)')
plt.grid(True,which = "both")
plt.legend()
plt.show()

# Visualization

In [ ]:
r_new_yearly = simulation.r_new * 365
r_old_weekly = simulation.r_old * 7

# Create a colormap
cmap = plt.cm.autumn_r

In [ ]:
#The log network (all connections in year)
adjacency_matrix = simulation.tot_log_matrix

# Step 2: Convert the sparse tensor to a list of edges
edges = adjacency_matrix._indices().t().tolist()

# Since it's bidirectional, add both directions for each edge
#bidirectional_edges = edges + [[b, a] for a, b in edges]

G_log = nx.Graph()
G_log.add_edges_from(edges)

# Extract the nodes present in the graph (those with at least one connection)
nodes_with_edges_log = list(G_log.nodes)
k = 0.13
pos_log = nx.spring_layout(G_log, k=k, iterations=2*int(np.sqrt(N)), seed=42)

## Yearly log

### Color wrt r_new

In [ ]:
# Map the node colors using r_new; assuming node indices match r_new indices
node_colors = [r_new_yearly[node].item() for node in nodes_with_edges_log]

# Step 4: Visualize the graph with node coloring based on r_new
plt.figure(figsize=(8, 6))

# Log-normalize node colors
log_norm = LogNorm(vmin=min(node_colors), vmax=max(node_colors))

# Map normalized values to colors using ScalarMappable
sm = ScalarMappable(cmap=cmap, norm=log_norm)
sm.set_array([])  # Needed for color bar

# Get the actual color values with 50% alpha
node_colors_mapped = [(*sm.to_rgba(color)[:3], 1) for color in node_colors]


# Draw the graph with the mapped colors and custom layout
# Adjust the width parameter to control edge thickness
nx.draw(
    G_log,
    pos_log,
    with_labels=False,
    node_color=node_colors_mapped,
    edge_color='gray',
    edgecolors = 'black',
    linewidths = 0.1, #Stroke thickness
    node_size=50,
    width= 0.3,  # Set the edge thickness here
    cmap=cmap
)

# Add colorbar, specifying the axis
cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical')
cbar.set_label('Partners per year',fontsize=14)

# Manually set ticks and labels for the colorbar
tick_positions = [0.5,1, 5, 10,50, 100, 200, 300]  # Customize these values as needed
tick_labels = ['0.5','1','5','10', '50', '100', '200','300']  # Corresponding labels
filtered_positions = [pos for pos in tick_positions if pos <= max(node_colors)]
filtered_labels = tick_labels[:len(filtered_positions)]
cbar.set_ticks(filtered_positions)
cbar.set_ticklabels(filtered_labels)

plt.title("Yearly log")
plt.show()


### Color wrt r_old

In [ ]:
# Map the node colors using r_new; assuming node indices match r_new indices
node_colors = [r_old_weekly[node].item() for node in nodes_with_edges_log]

# Step 4: Visualize the graph with node coloring based on r_new
plt.figure(figsize=(8, 6))

# Log-normalize node colors
log_norm = LogNorm(vmin=min(node_colors), vmax=max(node_colors))

# Map normalized values to colors using ScalarMappable
sm = ScalarMappable(cmap=cmap, norm=log_norm)
sm.set_array([])  # Needed for color bar

# Get the actual color values with 50% alpha
node_colors_mapped = [(*sm.to_rgba(color)[:3], 1) for color in node_colors]

# Draw the graph with the mapped colors and custom layout
# Adjust the width parameter to control edge thickness
nx.draw(
    G_log,
    pos_log,
    with_labels=False,
    node_color=node_colors_mapped,
    edge_color='gray',
    edgecolors = 'black',
    linewidths = 0.1,
    node_size=50,
    width= 0.3,  # Set the edge thickness here
    cmap=cmap
)

# Add colorbar, specifying the axis
cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical')
cbar.set_label('Network encounters pr week',fontsize = 15)

# Manually set ticks and labels for the colorbar
tick_positions = [0.5,1,2,3,4,5]  # Customize these values as needed
tick_labels = ['0.5','1','2','3','4','5']  # Corresponding labels
filtered_positions = [pos for pos in tick_positions if pos <= max(node_colors)]
filtered_labels = tick_labels[:len(filtered_positions)]
cbar.set_ticks(filtered_positions)
cbar.set_ticklabels(filtered_labels)

plt.title("Yearly log")
plt.show()


# Current network

In [ ]:
#The actual network
adjacency_matrix = network_matrix

# Step 2: Convert the sparse tensor to a list of edges
edges = adjacency_matrix._indices().t().tolist()

G_network = nx.Graph()
G_network.add_edges_from(edges)

# Extract the nodes present in the graph (those with at least one connection)
nodes_with_edges_network = list(G_network.nodes)

# Adjust the spring layout
pos_network = nx.spring_layout(G_network, k=k*12, iterations=2*int(np.sqrt(N)), seed=42)

In [ ]:
# Map the node colors using r_new; assuming node indices match r_new indices
node_colors = [r_new_yearly[node].item() for node in nodes_with_edges_network]

# Step 4: Visualize the graph with node coloring based on r_new
plt.figure(figsize=(8, 6))

# Log-normalize node colors
log_norm = LogNorm(vmin=min(node_colors), vmax=max(node_colors))

# Map normalized values to colors using ScalarMappable
sm = ScalarMappable(cmap=cmap, norm=log_norm)
sm.set_array([])  # Needed for color bar

# Get the actual color values with 50% alpha
node_colors_mapped = [(*sm.to_rgba(color)[:3], 1) for color in node_colors]

# Adjust the spring layout
pos_network = nx.spring_layout(G_network, k=0.13, iterations=50, seed=42)

# Draw the graph with the mapped colors and custom layout
# Adjust the width parameter to control edge thickness
nx.draw(
    G_network,
    pos_network,
    with_labels=False,
    node_color=node_colors_mapped,
    edge_color='gray',
    edgecolors = 'black',
    linewidths = 0.1,
    node_size=80,
    width= 0.3,  # Set the edge thickness here
    cmap=cmap
)

# Add colorbar, specifying the axis
cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical')
cbar.set_label('Partners per year',fontsize=15)

# Manually set ticks and labels for the colorbar
tick_positions = [0.5,1, 5, 10,50, 100, 200, 300]  # Customize these values as needed
tick_labels = ['0.5','1','5','10', '50', '100', '200','300']  # Corresponding labels
filtered_positions = [pos for pos in tick_positions if pos <= max(node_colors)]
filtered_labels = tick_labels[:len(filtered_positions)]
cbar.set_ticks(filtered_positions)
cbar.set_ticklabels(filtered_labels)


plt.title("Current network")
plt.show()


## Show evolving network

In [ ]:
# Initial visualization setup
indices_sparse = torch.sparse_coo_tensor(indices, torch.ones(indices.shape[1]))
indices_sparse = indices_sparse.coalesce()
network_indices = indices_sparse.indices()
strengths = indices_sparse.values()
network_matrix = torch.sparse_coo_tensor(indices=network_indices, values=strengths, size=(N, N))
network_indices_transposed = torch.stack([network_indices[1], network_indices[0]])
network_matrix_T = torch.sparse_coo_tensor(indices=network_indices_transposed, values=strengths, size=(N, N))
network_matrix = network_matrix + network_matrix_T
adjacency_matrix = network_matrix


# Visualization loop
plt.ion()  # Turn on interactive mode for dynamic updates
fig, ax = plt.subplots(figsize=(8, 6))
t = tot_log[2].max()
indices2,seeking2 = copy.deepcopy(indices),copy.deepcopy(seeking)
for _ in range(100):  # Number of iterations, adjust as needed
    # Run one iteration of the simulation
    indices2, seeking2, _, _, _, _, _, _, _, _, _, _, _, _ = Iterate(
        N, M, indices2, seeking2, r_old, r_new, False, tot_log, t
    )
    t += 1
    # Update the network with new indices
    indices_sparse = torch.sparse_coo_tensor(indices, torch.ones(indices.shape[1]))
    indices_sparse = indices_sparse.coalesce()
    network_indices = indices_sparse.indices()
    strengths = indices_sparse.values()
    network_matrix = torch.sparse_coo_tensor(indices=network_indices, values=strengths, size=(N, N))
    network_indices_transposed = torch.stack([network_indices[1], network_indices[0]])
    network_matrix_T = torch.sparse_coo_tensor(indices=network_indices_transposed, values=strengths, size=(N, N))
    network_matrix = network_matrix + network_matrix_T
    adjacency_matrix = network_matrix

    # Convert the sparse tensor to a list of edges
    edges = adjacency_matrix._indices().t().tolist()

    # Update the graph with new edges
    G_network.clear()
    G_network.add_edges_from(edges)

    # Use the previous positions as the initial guess
    pos_network = nx.spring_layout(G_network, pos=pos_network, k=k, iterations=20, seed=42)

    # Map node colors based on r_new_yearly
    node_colors = [r_new_yearly[node].item() for node in G_network.nodes]

    # Log-normalize node colors
    log_norm = LogNorm(vmin=min(node_colors), vmax=max(node_colors))
    sm = ScalarMappable(cmap=cmap, norm=log_norm)
    sm.set_array([])

    node_colors_mapped = [(*sm.to_rgba(color)[:3], 1) for color in node_colors]

    ax.clear()  # Clear the axis for the next frame

    # Draw the graph
    nx.draw(
        G_network,
        pos_network,
        with_labels=False,
        node_color=node_colors_mapped,
        edge_color='gray',
        edgecolors='black',
        linewidths=0.1,
        node_size=80,
        width=0.3,
        cmap=cmap,
        ax=ax
    )

    # Add colorbar
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical')
    cbar.set_label('Partners per year')
    cbar.set_ticks([0.5, 1, 5])
    cbar.set_ticklabels(['0.5', '1', '5'])

    ax.set_title("Current network")
    plt.pause(0.1)  # Pause for a short time to update the plot

plt.ioff()  # Turn off interactive mode
plt.show()


### Color wrt r_old

In [ ]:
# Map the node colors using r_new; assuming node indices match r_new indices
node_colors = [r_old_weekly[node].item() for node in nodes_with_edges_network]

# Step 4: Visualize the graph with node coloring based on r_new
plt.figure(figsize=(8, 6))

# Log-normalize node colors
log_norm = LogNorm(vmin=min(node_colors), vmax=max(node_colors))

# Map normalized values to colors using ScalarMappable
sm = ScalarMappable(cmap=cmap, norm=log_norm)
sm.set_array([])  # Needed for color bar

# Get the actual color values with 50% alpha
node_colors_mapped = [(*sm.to_rgba(color)[:3], 1) for color in node_colors]

# Adjust the spring layout
pos_network = nx.spring_layout(G_network, k=0.13, iterations=50, seed=42)

# Draw the graph with the mapped colors and custom layout
# Adjust the width parameter to control edge thickness
nx.draw(
    G_network,
    pos_network,
    with_labels=False,
    node_color=node_colors_mapped,
    edge_color='gray',
    edgecolors = 'black',
    linewidths = 0.1,
    node_size=80,
    width= 0.3,  # Set the edge thickness here
    cmap=cmap
)

# Add colorbar, specifying the axis
cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical')
cbar.set_label('Network encounters per week')

# Manually set ticks and labels for the colorbar
tick_positions = [0.5, 1, 5]  # Customize these values as needed
tick_labels = ['0.5','1','5']  # Corresponding labels
cbar.set_ticks(tick_positions)
cbar.set_ticklabels(tick_labels)


plt.title("Current network")
plt.show()
